In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [2]:
온라인 = pd.read_csv('온라인_전처리_final_32columns.csv')
print(온라인.shape)
온라인.head()

(3196362, 32)


,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


In [4]:
# 각 clnt_id별 session이 바뀌는 지점 index 저장
idx1 = 온라인.unique_id.drop_duplicates().index.tolist()
idx2 = idx1[1:] + [len(온라인)]

In [5]:
# 구매 시점이 있으면 index 저장하고 없으면 마지막 index 그대로 가져옴
idx3 = []
for i, j in tqdm_notebook(zip(idx1, idx2), total=len(idx1)):
    temp = 온라인.buy.iloc[i:j]
    try:
        idx3.append(temp[temp == 1].index[0])
    except:
        idx3.append(j)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
# 첫번째 구매시점 기준으로 제거함
temp = []
for i, j in tqdm_notebook(zip(idx1, idx3), total=len(idx1)):
    temp.append(온라인.iloc[i:j].values)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
온라인전체 = pd.DataFrame(list(itertools.chain.from_iterable(temp)))
온라인전체.columns = 온라인.columns
print(온라인전체.shape)
온라인전체.head()

(3030358, 32)


,clnt_id,sess_id,trans_id,hit_seq,hit_pss_tm,time_length,holiday_diff,keyword,action_type_0,action_type_1,...,trfc_src_PORTAL_3,trfc_src_PUSH,trfc_src_WEBSITE,trfc_src_unknown,dvc_ctg_nm_PC,dvc_ctg_nm_mobile_app,dvc_ctg_nm_mobile_web,dvc_ctg_nm_unknown,unique_id,buy
0,1,1,NaN,1,12.0,11.0,0,1,1,0,...,0,0,0,1,0,1,0,0,1_1,0
1,1,1,NaN,2,23.0,14.0,0,2,1,0,...,0,0,0,1,0,1,0,0,1_1,0
2,1,1,NaN,3,37.0,14.0,0,3,1,0,...,0,0,0,1,0,1,0,0,1_1,0
3,1,2,NaN,1,42.0,15.0,1,1,1,0,...,0,0,0,1,0,1,0,0,1_2,0
4,1,2,NaN,2,57.0,14.0,1,2,1,0,...,0,0,0,1,0,1,0,0,1_2,0


In [50]:
# 온라인전체 (첫구매 이전) 변수별 평균값
temp = 온라인전체.copy()
temp['hit_seq_num'] = temp.hit_seq
temp.groupby('hit_seq_num').mean().iloc[:10,3:-1]
temp.to_csv('온라인전체.csv',index=False)

In [47]:
# 구매/비구매 unique_id (clnt_id + sess_id) 추출
구매여부 = 온라인.groupby('unique_id').buy.sum()
구매 = 구매여부[구매여부>0].index.tolist()
비구매 = 구매여부[구매여부==0].index.tolist()

In [51]:
# 온라인 구매 (첫구매 이전) 변수별 평균값
온라인구매 = 온라인전체[온라인전체.unique_id.isin(구매)]
temp = 온라인구매.copy()
temp['hit_seq_num'] = temp.hit_seq
temp.groupby('hit_seq_num').mean().iloc[:10,3:-1]
temp.to_csv('온라인구매.csv',index=False)

In [52]:
# 온라인 비구매 변수별 평균값
온라인비구매 = 온라인전체[온라인전체.unique_id.isin(비구매)]
temp = 온라인비구매.copy()
temp['hit_seq_num'] = temp.hit_seq
temp.groupby('hit_seq_num').mean().iloc[:10,3:-1]
temp.to_csv('온라인비구매.csv',index=False)